<a href="https://colab.research.google.com/github/chhyyi/aiffel/blob/main/LMS/nlp_node8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About this notebook  
이 노트북은 모두의 연구소 AIFFEL LMS NLP node8의 프로젝트입니다. seq2seq 한-영 번역기를 attention을 사용하여 만듭니다.   

This is written as a part of the deep learning training course by Aiffel, Korea. LMS NLP node 8. I have to make a korean - english seq2seq translator. Additionaly attention should be used.

In [18]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

# Checkout version of libraries

In [19]:
import pandas as pd
import matplotlib

print(pd.__version__)
print(tf.__version__)
print(matplotlib.__version__)

1.3.5
2.9.2
3.2.2


# Download, load dataset
download korean-english-park.train.tar.gz from this github rep.
[jungyeul/korean-parallel-corpora](https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1)  
instead of tf.keras.utils.get_file(), google drive web page and terminal used.

In [22]:
en_path = '/content/drive/MyDrive/colabdata/modulabs/nlp8/korean-english-park.train.en'
ko_path = '/content/drive/MyDrive/colabdata/modulabs/nlp8/korean-english-park.train.ko'
f_ko=open(ko_path, 'r')
f_en=open(en_path, 'r')
ko_raw = f_ko.readlines()
en_raw = f_en.readlines()

In [23]:
print(len(ko_raw), len(en_raw))

94123 94123


In [25]:
data = pd.DataFrame()
test_rows=5000 #temporal index for scripting
if test_rows:
    data['ko_raw']=ko_raw[:test_rows]
    data['en_raw']=en_raw[:test_rows]
else:
    data['ko_raw']=ko_raw
    data['en_raw']=en_raw
data

,ko_raw,en_raw
0,"개인용 컴퓨터 사용의 상당 부분은 ""이것보다 뛰어날 수 있느냐?""\n","Much of personal computing is about ""can you t..."
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,so a mention a few weeks ago about a rechargea...
2,그러나 이것은 또한 책상도 필요로 하지 않는다.\n,"Like all optical mice, But it also doesn't nee..."
3,"79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분...",uses gyroscopic sensors to control the cursor ...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔...,Intelligence officials have revealed a spate o...
...,...,...
4995,여러분의 이웃부터 시작하여 여러분의 국가에 봉사해 주기를 말입니다.\n,"to serve your nation, beginning with your neig..."
4996,방관자가 아니라 시민이;\n,I ask you to be citizens:\n
4997,피통치자가 아니라 시민이;\n,"citizens, not spectators;\n"
4998,"봉사하는 사회, 그리고 기개가 있는 국가를 건설하는 책임있는 시민이 되기를 말입니다.\n","responsible citizens, building communities of ..."


# About Data load...

두 파일은 영어, 한국어 문장이 순서대로 들어가있으므로 순서를 그대로 읽어야 합니다. 
pandas.read_csv나 read_table을 썼을 때에는 줄의 개수가 달라져서 seperator를 고쳐보다가 안돼서 그냥 file.readlines()를 사용하였습니다.

Two data file have corresponding sentences. So its order should be preserved. but when I have tried pandas.read_csv and pandas.read_table first. In that case, length of both file did not matched. maybe I should change seperator   



# Data refine, cleaning

In [26]:
data.isna().any()

ko_raw    False
en_raw    False
dtype: bool

In [27]:
data.shape

(5000, 2)

## Characters?
 before we process regular expressions, let's figure out set of characters shown in each dataset.

In [42]:
from functools import reduce
def characters_in_column(df, col_name):
    """characters_in_column(df, col_name)
    df is pandas dataframe. this function converts df["col_name"] into set of characters. return ordered list.
    """
    _=pd.Series()
    _=df[col_name].apply(lambda x: [i for i in x])
    return np.unique(np.concatenate(_.values))


In [51]:
ko_chars=characters_in_column(data, 'ko_raw')
print(ko_chars)
print(' '.join(ko_chars))

['\n' ' ' '!' ... '率' '識' '？']

   ! " % & ' ( ) * , - . / 0 1 2 3 4 5 6 7 8 9 : ; = ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ ] _ a b c d e f g h i j k l m n o p q r s t u v w x y z   ― ‘ ’ … 三 不 之 九 亞 京 修 傷 像 光 內 兵 再 動 勝 北 卿 反 受 古 同 名 命 善 喩 園 在 型 城 基 多 大 天 奔 奴 字 安 家 富 察 寧 寶 對 島 峽 州 市 席 店 庫 建 心 性 愛 懷 戰 手 掌 故 敎 敢 文 新 方 族 旗 日 早 明 會 朱 案 棺 植 機 權 殘 毁 母 民 水 江 沈 法 泥 流 海 淘 湖 湛 溫 澤 濤 灣 無 煙 父 物 獸 率 現 生 産 異 畵 疑 登 省 社 私 種 空 章 紙 纏 置 美 群 老 耳 聖 聲 膚 舊 花 英 華 董 蓋 號 街 衛 要 親 言 誌 論 諸 識 質 赤 超 足 路 身 軍 迷 道 邸 都 酸 銅 錦 鎔 長 門 開 陳 難 霧 靑 頂 頭 題 食 駐 體 髮 鼓 가 각 간 갇 갈 감 갑 값 갓 갔 강 갖 갗 같 갚 갛 개 객 갤 갱 거 걱 건 걷 걸 검 겁 것 겉 게 겐 겟 겠 겨 격 겪 견 결 겸 겹 겼 경 곁 계 고 곡 곤 곧 골 곰 곱 곳 공 과 곽 관 괄 괌 광 괘 괜 괭 괴 교 구 국 군 굳 굴 굶 굽 굿 궁 권 궐 궤 귀 규 균 그 극 근 글 금 급 긋 긍 기 긴 길 김 깃 깅 깊 까 깎 깐 깔 깜 깝 깡 깥 깨 깼 꺼 껌 껍 껏 껑 께 껴 꼈 꼬 꼭 꼼 꼽 꼿 꽁 꽂 꽃 꽉 꽝 꽤 꽥 꾸 꾼 꿀 꿈 꿔 꿨 꿰 뀌 끄 끈 끊 끌 끓 끔 끗 끝 끼 낀 낄 낌 낍 나 낙 낚 난 날 낡 남 납 낫 났 낭 낮 낯 낳 내 낸 낼 냄 냅 냈 냉 냐 냥 너 넌 널 넓 넘 넣 네 넥 넨 넬 넴 넷 녀 녁 년 념 녔 노 녹 논 놀 농 높 놓 놨 뇌 뇨 누 눅 눈 눌 눕 눴 뉘 뉴 늄 늉 느 늑 는 늘 늙 능 늦 늬 니 닉 닌 닐 님 닙 닛 닝 다 닥 단 닫 달 닭 닮

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


온갖 글자가 다 있습니다. 줄넘김, 한자도 있네요.

## 텍스트 전처리 수정(정제)
노드의 코드를 보면 정규표현식을 다음과 같이 사용합니다.
```python
sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
sentence = re.sub(r'[" "]+', " ", sentence)
sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
```
위의 예에서 보면 많은 경우 \n이 삽입돼 있는데 줄넘김 표시이므로 삭제합니다.  
외래어도 자연스럽게 우리말 문장에 영어로 들어가 있는 경우도 많 때문에 굳이 영어 알파벳를 빼지는 않겠습니다.

```python
sentence = re.sub(r"[\n]", " ", sentence)
sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
sentence = re.sub(r'[" "]+', " ", sentence)
sentence = re.sub(r"[^ㄱ-ㅎ가-힣a-zA-Z?.!,]+", " ", sentence)
```

In [56]:
def preprocess_en(sentence, s_token=True, e_token=True):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

In [57]:
def preprocess_ko(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"[\n]", " ", sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^ㄱ-ㅎ가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [59]:
data['ko_sentence']=data['ko_raw'].apply(preprocess_ko)
data['en_sentence']=data['en_raw'].apply(preprocess_en)
data

,ko_raw,en_raw,ko_sentence,en_sentence
0,"개인용 컴퓨터 사용의 상당 부분은 ""이것보다 뛰어날 수 있느냐?""\n","Much of personal computing is about ""can you t...",개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?,<start> much of personal computing is about ca...
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,so a mention a few weeks ago about a rechargea...,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,<start> so a mention a few weeks ago about a r...
2,그러나 이것은 또한 책상도 필요로 하지 않는다.\n,"Like all optical mice, But it also doesn't nee...",그러나 이것은 또한 책상도 필요로 하지 않는다 .,"<start> like all optical mice , but it also do..."
3,"79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분...",uses gyroscopic sensors to control the cursor ...,". 달러하는 이 최첨단 무선 광마우스는 허공에서 팔목 , 팔 , 그외에 어떤 부분이...",<start> uses gyroscopic sensors to control the...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔...,Intelligence officials have revealed a spate o...,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 테러 계획들이 실패로 돌아갔음을...,<start> intelligence officials have revealed a...
...,...,...,...,...
4995,여러분의 이웃부터 시작하여 여러분의 국가에 봉사해 주기를 말입니다.\n,"to serve your nation, beginning with your neig...",여러분의 이웃부터 시작하여 여러분의 국가에 봉사해 주기를 말입니다 .,"<start> to serve your nation , beginning with ..."
4996,방관자가 아니라 시민이;\n,I ask you to be citizens:\n,방관자가 아니라 시민이,<start> i ask you to be citizens <end>
4997,피통치자가 아니라 시민이;\n,"citizens, not spectators;\n",피통치자가 아니라 시민이,"<start> citizens , not spectators <end>"
4998,"봉사하는 사회, 그리고 기개가 있는 국가를 건설하는 책임있는 시민이 되기를 말입니다.\n","responsible citizens, building communities of ...","봉사하는 사회 , 그리고 기개가 있는 국가를 건설하는 책임있는 시민이 되기를 말입니다 .","<start> responsible citizens , building commun..."


In [60]:
data.drop(columns=['en_raw','ko_raw'], inplace=True)
data

,ko_sentence,en_sentence
0,개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?,<start> much of personal computing is about ca...
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,<start> so a mention a few weeks ago about a r...
2,그러나 이것은 또한 책상도 필요로 하지 않는다 .,"<start> like all optical mice , but it also do..."
3,". 달러하는 이 최첨단 무선 광마우스는 허공에서 팔목 , 팔 , 그외에 어떤 부분이...",<start> uses gyroscopic sensors to control the...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 테러 계획들이 실패로 돌아갔음을...,<start> intelligence officials have revealed a...
...,...,...
4995,여러분의 이웃부터 시작하여 여러분의 국가에 봉사해 주기를 말입니다 .,"<start> to serve your nation , beginning with ..."
4996,방관자가 아니라 시민이,<start> i ask you to be citizens <end>
4997,피통치자가 아니라 시민이,"<start> citizens , not spectators <end>"
4998,"봉사하는 사회 , 그리고 기개가 있는 국가를 건설하는 책임있는 시민이 되기를 말입니다 .","<start> responsible citizens , building commun..."


## 토큰화 수정
한국어의 형태소 분석 적용 토큰화: MeCab을 사용합니다. 이것은 [exploration node 8](https://colab.research.google.com/drive/1lv3FDw2hKe_Gp56mzCxGPIHof8p0m6Cz?usp=sharing)에서 했던 것을 그대로 가져옵니다.


In [46]:
!apt-get update && apt-get install -y g++ default-jdk
!pip install konlpy
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,208 kB]
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/machine

In [ ]:
import konlpy
import gensim
from konlpy.tag import Mecab

num_words=10000
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']